In [14]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [15]:
content=pd.read_csv('data/Content.csv',chunksize=5000)

In [19]:
for i in content:
    i=i.loc[i['source_type']=='Compound']
    if len(i)>0:
        display(i)
        break

,id,source_id,source_type,food_id,orig_food_id,orig_food_common_name,orig_food_scientific_name,orig_food_part,orig_source_id,orig_source_name,orig_content,orig_min,orig_max,orig_unit,orig_citation,citation,citation_type,creator_id,updater_id,created_at,updated_at,orig_method,orig_unit_expression,standard_content,preparation_type,export
116684,128665,21594,Compound,287,868,Alfalfa seeds,NaN,NaN,4,"Fatty acids, total saturated",0.000,NaN,NaN,mg/100g,NaN,DTU,DATABASE,NaN,NaN,2014-11-05 14:54:39 UTC,2019-12-11 20:28:23 UTC,NaN,NaN,0.000,other,0
116685,128666,21595,Compound,287,868,Alfalfa seeds,NaN,NaN,5,"Fatty acids, total mono-unsaturated",0.000,NaN,NaN,mg/100g,NaN,DTU,DATABASE,NaN,NaN,2014-11-05 14:54:39 UTC,2019-12-11 20:28:23 UTC,NaN,NaN,0.000,other,0
116686,128667,21595,Compound,287,868,Alfalfa seeds,NaN,NaN,6,"Fatty acids, total poly-unsaturated",0.000,NaN,NaN,mg/100g,NaN,DTU,DATABASE,NaN,NaN,2014-11-05 14:54:39 UTC,2019-12-11 20:28:23 UTC,NaN,NaN,0.000,other,0
116687,128668,1131,Compound,287,868,Alfalfa seeds,NaN,NaN,9,"Carbohydrates, added sugar (sucrose)",0.000,NaN,NaN,mg/100g,NaN,DTU,DATABASE,NaN,NaN,2014-11-05 14:54:39 UTC,2019-12-11 20:28:23 UTC,NaN,NaN,0.000,other,0
116688,128669,753,Compound,287,868,Alfalfa seeds,NaN,NaN,11,Alcohol (ethanol),0.000,NaN,NaN,mg/100g,NaN,DTU,DATABASE,NaN,NaN,2014-11-05 14:54:39 UTC,2019-12-11 20:28:23 UTC,NaN,NaN,0.000,other,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119995,131976,13393,Compound,200,817,"Bean sprouts, mung, raw",NaN,NaN,13,Moisture,90400.000,NaN,NaN,mg/100g,NaN,DTU,DATABASE,NaN,NaN,2014-11-05 14:55:31 UTC,2019-12-11 20:28:34 UTC,NaN,NaN,90400.000,raw,0
119996,131977,13831,Compound,200,817,"Bean sprouts, mung, raw",NaN,NaN,14,"Vitamin A, total",1.000,NaN,NaN,RE,NaN,DTU,DATABASE,NaN,NaN,2014-11-05 14:55:31 UTC,2019-12-11 20:28:34 UTC,NaN,NaN,1.000,raw,0
119997,131978,13831,Compound,200,817,"Bean sprouts, mung, raw",NaN,NaN,15,Vitamin A (retinol),0.000,NaN,NaN,mg/100g,NaN,DTU,DATABASE,NaN,NaN,2014-11-05 14:55:31 UTC,2019-12-11 20:28:34 UTC,NaN,NaN,0.000,raw,0
119998,131979,14616,Compound,200,817,"Bean sprouts, mung, raw",NaN,NaN,16,"Carotene, beta",0.012,NaN,NaN,mg/100g,NaN,DTU,DATABASE,NaN,NaN,2014-11-05 14:55:31 UTC,2019-12-11 20:28:34 UTC,NaN,NaN,0.012,raw,0


In [45]:
compound=pd.read_csv('data/Compound.csv')

/Users/zhaoxiangchen/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (3,4,5,6,12,13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [46]:
compound.head(10)

,id,public_id,name,moldb_iupac,state,annotation_quality,description,cas_number,moldb_inchikey,moldb_inchi,moldb_smiles,moldb_mono_mass,kingdom,superklass,klass,subklass
0,4,FDB000004,Cyanidin 3-(6''-acetyl-galactoside),NaN,low,Constituent of the leaves of Nymphaea alba [CC...,350602-26-5,[H][C@]1(COC(C)=O)O[C@@]([H])(OC2=CC3=C(O)C=C(...,InChI=1S/C23H22O12/c1-9(24)32-8-18-19(29)20(30...,491.118951,HBXXDBKJLPLXPR-DLBZZEGUSA-O,"3-{[(2S,3R,4S,5R,6R)-6-[(acetyloxy)methyl]-3,4...",Organic compounds,Phenylpropanoids and polyketides,Flavonoids,Flavonoid glycosides
1,13,FDB000013,Cyanidin 3-(6''-succinyl-glucoside),NaN,low,Constituent of Phragmites australis [CCD]. Cya...,216692-08-9,[H][C@]1(COC(=O)CCC(O)=O)O[C@@]([H])(OC2=CC3=C...,InChI=1S/C25H24O14/c26-11-6-14(28)12-8-17(24(3...,549.124431,MIYGQTFETYBMKF-WVXUANQFSA-O,"7-methoxy-2,2-dimethyl-2H-chromene-6-carboxyli...",Organic compounds,Phenylpropanoids and polyketides,Flavonoids,Flavonoid glycosides
2,14,FDB000014,Pelargonidin 3-(6''-succinyl-glucoside),liquid,low,Pelargonidin 3-(6''-succinyl-glucoside) is a m...,NaN,[H][C@]1(COC(=O)CCC(O)=O)O[C@@]([H])(OC2=CC3=C...,InChI=1S/C25H24O13/c26-12-3-1-11(2-4-12)24-17(...,533.129516,UBUSYXLSGMWUJJ-WVXUANQFSA-O,"3-{[(2S,3R,4S,5S,6R)-6-{[(3-carboxypropanoyl)o...",NaN,NaN,NaN,NaN
3,24,FDB000024,Petunidin 3-O-(6''-acetyl-galactoside),NaN,low,Petunidin 3-o-(6''-acetyl-galactoside) is a me...,NaN,[H][C@]1(COC(C)=O)OC(OC2=C([O+]=C3C=C(O)C=C(O)...,InChI=1S/C24H24O13/c1-9(25)34-8-18-20(30)21(31...,521.129516,GPUBWXUQPURXOQ-SKKXNPCDSA-O,"3-{[(3R,4S,5R,6R)-6-[(acetyloxy)methyl]-3,4,5-...",NaN,NaN,NaN,NaN
4,25,FDB000025,Peonidin 3-(6''-acetyl-galactoside),NaN,low,Peonidin 3-(6''-acetyl-galactoside) is a membe...,75-07-0,[H][C@]1(COC(C)=O)OC(OC2=C([O+]=C3C=C(O)C=C(O)...,InChI=1S/C24H24O12/c1-10(25)33-9-19-20(29)21(3...,505.134601,MBSKDCPWFSMEFD-ZKVZURMCSA-O,"3-{[(3R,4S,5R,6R)-6-[(acetyloxy)methyl]-3,4,5-...",NaN,NaN,NaN,NaN
5,27,FDB000027,Malvidin 3-(6''-acetyl-galactoside),NaN,low,Malvidin 3-(6''-acetyl-galactoside) is a membe...,75-07-0,[H][C@]1(COC(C)=O)OC(OC2=C([O+]=C3C=C(O)C=C(O)...,InChI=1S/C25H26O13/c1-10(26)35-9-19-21(30)22(3...,535.145166,WGYWEDJQQFKGID-ZTKYFMRJSA-O,"3-{[(3R,4S,5R,6R)-6-[(acetyloxy)methyl]-3,4,5-...",NaN,NaN,NaN,NaN
6,31,FDB000031,Pelargonidin 3-arabinoside,liquid,low,Pelargonidin 3-arabinoside is a member of the ...,NaN,[H][C@]1(CO)O[C@]([H])(OC2=CC3=C(O)C=C(O)C=C3[...,InChI=1S/C20H18O9/c21-8-16-17(25)18(26)20(29-1...,403.102907,BGOQMKHOLJPANL-XSYGEPLQSA-O,"3-{[(2R,3S,4S,5R)-3,4-dihydroxy-5-(hydroxymeth...",NaN,NaN,NaN,NaN
7,35,FDB000035,Peonidin 3-(6''-p-coumaroyl-glucoside),NaN,low,Peonidin 3-(6''-p-coumaroyl-glucoside) is a me...,147-85-3,[H][C@]1(COC(=O)\C=C\C2=CC=C(O)C=C2)OC(OC2=C([...,InChI=1S/C31H28O13/c1-40-23-10-16(5-8-20(23)34...,609.160816,MFHMTMFSFNMTFQ-RJBRRTDCSA-O,"5,7-dihydroxy-2-(4-hydroxy-3-methoxyphenyl)-3-...",NaN,NaN,NaN,NaN
8,40,FDB000040,Delphinidin 3-xyloside,NaN,low,Delphinidin 3-xyloside is a member of the clas...,NaN,[H][C@]1(CO)O[C@@]([H])(OC2=C([O+]=C3C=C(O)C=C...,InChI=1S/C20H18O11/c21-6-15-17(27)18(28)20(31-...,435.092736,WIEYMFHXYNRELM-MJKGWSOKSA-O,"3-{[(2S,3R,4R,5R)-3,4-dihydroxy-5-(hydroxymeth...",NaN,NaN,NaN,NaN
9,41,FDB000041,Petunidin 3-(6''-p-coumaroyl-glucoside),NaN,low,Petunidin 3-(6''-p-coumaroyl-glucoside) is a m...,147-85-3,[H][C@]1(COC(=O)\C=C\C2=CC=C(O)C=C2)OC(OC2=C([...,InChI=1S/C31H28O14/c1-41-22-9-15(8-20(35)26(22...,625.155731,KTFQEFWNLAUGAX-YQUABMASSA-O,"2-(3,4-dihydroxy-5-methoxyphenyl)-5,7-dihydrox...",NaN,NaN,NaN,NaN


In [24]:
foodb=pd.read_csv('data/Drug_info_combined.csv')

In [56]:
foodb

,Unnamed: 0,Drug name,Name,Smiles
0,0,DB00006,Bivalirudin,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...
1,1,DB00014,Goserelin,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...
2,2,DB00035,Desmopressin,NC(=O)CC[C@@H]1NC(=O)[C@H](CC2=CC=CC=C2)NC(=O)...
3,3,DB00050,Cetrorelix,CC(C)C[C@H](NC(=O)[C@@H](CCCNC(N)=O)NC(=O)[C@H...
4,4,DB00080,Daptomycin,CCCCCCCCCC(=O)N[C@@H](CC1=CNC2=C1C=CC=C2)C(=O)...
...,...,...,...,...
2381,2381,DB14104,NaN,CCCCC\C=C/C\C=C/CCCCCCCC(O)=O
2382,2382,DB14116,NaN,O.[O--].[O--].[Mg++].[Mg++].[Al+3].[Al+3].[O-]...
2383,2383,DB14151,NaN,[P]
2384,2384,DB14156,NaN,CC1(C)C2CCC1(C)C(=O)C2


In [57]:
foodb.merge(compound,left_on='Smiles',right_on='cas_number')

,Unnamed: 0,Drug name,Name,Smiles,id,public_id,name,moldb_iupac,state,annotation_quality,description,cas_number,moldb_inchikey,moldb_inchi,moldb_smiles,moldb_mono_mass,kingdom,superklass,klass,subklass
0,9,DB00114,Pyridoxal Phosphate,CC1=NC=C(COP(O)(O)=O)C(C=O)=C1O,21978,FDB021820,Pyridoxal 5'-phosphate,Solid,low,This is the active form of vitamin B6 serving ...,54-47-7,CC1=NC=C(COP(O)(O)=O)C(C=O)=C1O,InChI=1S/C8H10NO6P/c1-5-8(11)7(3-10)6(2-9-5)4-...,247.024574,NGVDGCNFYWLIFO-UHFFFAOYSA-N,[(4-formyl-5-hydroxy-6-methylpyridin-3-yl)meth...,Organic compounds,Organoheterocyclic compounds,Pyridines and derivatives,Pyridine carboxaldehydes
1,13,DB00119,Pyruvic acid,CC(=O)C(O)=O,31304,FDB031141,pyruvate,NaN,NaN,"Pyruvic acid, also known as 2-oxopropanoic aci...",127-17-3,CC(=O)C(O)=O,"InChI=1S/C3H4O3/c1-2(4)3(5)6/h1H3,(H,5,6)",88.016044,LCTONWCANYUPML-UHFFFAOYSA-N,2-oxopropanoic acid,NaN,NaN,NaN,NaN
2,14,DB00120,L-Phenylalanine,N[C@@H](CC1=CC=CC=C1)C(O)=O,4941,FDB004940,L-(-)-Phenylalanine,NaN,low,"L-phenylalanine, also known as phe or f, belon...",63-91-2,N[C@@H](CC1=CC=CC=C1)C(O)=O,InChI=1S/C9H11NO2/c10-8(9(11)12)6-7-4-2-1-3-5-...,165.078979,COLNVLDHVKWLRT-QMMMGPOBSA-N,(2S)-2-amino-3-phenylpropanoic acid,NaN,NaN,NaN,NaN
3,16,DB00122,Choline,C[N+](C)(C)CCO,710,FDB000710,Choline,Solid,low,Nutritional supplement. Occurs free and combin...,62-49-7,C[N+](C)(C)CCO,"InChI=1S/C5H14NO/c1-6(2,3)4-5-7/h7H,4-5H2,1-3H...",104.107539,OEYIOHPDSNJKLS-UHFFFAOYSA-N,(2-hydroxyethyl)trimethylazanium,Organic compounds,Organonitrogen compounds,Quaternary ammonium salts,Cholines
4,17,DB00123,L-Lysine,NCCCC[C@H](N)C(O)=O,474,FDB000474,L-Lysine,Solid,low,"Dietary supplement, nutrient. Found widely in ...",56-87-1,NCCCC[C@H](N)C(O)=O,"InChI=1S/C6H14N2O2/c7-4-2-1-3-5(8)6(9)10/h5H,1...",146.105528,KDXKERNSBIXSRK-YFKPBYRVSA-N,"(2S)-2,6-diaminohexanoic acid",Organic compounds,Organic acids and derivatives,Carboxylic acids and derivatives,"Amino acids, peptides, and analogues"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,2370,DB13977,NaN,[K+].[K+].[O-]C([O-])=O,15416,FDB015412,Potassium carbonate (K2CO3),NaN,low,"Food flavouring agent and adjuvant, pH control...",584-08-7,[K+].[K+].[O-]C([O-])=O,"InChI=1S/CH2O3.2K/c2-1(3)4;;/h(H2,2,3,4);;/q;2...",137.912158,BWHMMNNQKKPAPP-UHFFFAOYSA-L,dipotassium carbonate,NaN,NaN,NaN,NaN
345,2375,DB14015,NaN,[Na+].OS([O-])=O,15424,FDB015420,Sodium hydrogen sulfite,NaN,low,It is used in food processing as sanitising ag...,7631-90-5,[Na+].OS([O-])=O,"InChI=1S/Na.H2O3S/c;1-4(2)3/h;(H2,1,2,3)/q+1;/p-1",103.954409,DWAQJAXMDSEUJJ-UHFFFAOYSA-M,sodium hydrogen sulfite,NaN,NaN,NaN,NaN
346,2377,DB14028,NaN,ClC1=CC=C2NC(=O)CN=C(C3=CC=CC=C3)C2=C1,7125,FDB007124,N-Demethyldiazepam,NaN,low,"N-demethyldiazepam, also known as nordiazepam ...",1088-11-5,ClC1=CC=C2NC(=O)CN=C(C3=CC=CC=C3)C2=C1,InChI=1S/C15H11ClN2O/c16-11-6-7-13-12(8-11)15(...,270.055991,AKPLHCDWDRPJGD-UHFFFAOYSA-N,"7-chloro-5-phenyl-2,3-dihydro-1H-1,4-benzodiaz...",NaN,NaN,NaN,NaN
347,2381,DB14104,NaN,CCCCC\C=C/C\C=C/CCCCCCCC(O)=O,6563,FDB006562,Linonelic acid,NaN,low,"Linoleate, also known as (9z,12z)-octadecadien...",NaN,CCCCC\C=C/C\C=C/CCCCCCCC(O)=O,InChI=1S/C18H32O2/c1-2-3-4-5-6-7-8-9-10-11-12-...,280.240230,OYHQOLUKZRVURQ-HZJYTTRNSA-N,"(9Z,12Z)-octadeca-9,12-dienoic acid",NaN,NaN,NaN,NaN


In [58]:
!pip install cirpy

  Created wheel for cirpy: filename=CIRpy-1.0.2-py3-none-any.whl size=7267 sha256=8c31a6a759d9593a2fd03e7d7c2e84313e2ae6b5c762cacc6c605d81946e6257
  Stored in directory: /Users/zhaoxiangchen/Library/Caches/pip/wheels/36/03/bd/faf3adf70cc88b12abb0923de9c5c4cec57044730dba01ef7b
Successfully built cirpy
